In [ ]:
import sys
import os

if 'root_dir' not in globals():
    # rootディレクトリへのパスを設定
    root_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
    os.chdir(root_dir)

# 日時の取得
from datetime import datetime
now = datetime.now()

import logging
# ロギングの設定
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
os.getcwd()

In [ ]:
import pandas as pd
import pickle

search_word = '農薬'
date = '2024-07-05 12:35:17.161253'

data = pd.read_pickle(f'data/topic_dist_{search_word}_{date}.pkl')

In [ ]:
#データベースへ接続するエンジンを作成
from my_codes.database_setting import Engine
from my_codes.database_setting import Base

#データベースのテーブルとマッピングする
from my_codes.notes_database import Notes

from sqlalchemy.orm import sessionmaker
from sqlalchemy import func

In [ ]:
data.head()

In [ ]:
# 特定のトピックの割合が高い文書を抽出，キーを取り出す．
topic_id = 10
top_n = 50
top_docs = data.sort_values(by=topic_id, ascending=False).head(top_n)
top_keys = top_docs['key'].tolist()
#top_keys

In [ ]:
# top_keysに含まれるキーを持つ文書をデータベースから取得
Session = sessionmaker(bind=Engine)
session = Session()

result = session.query(Notes.key,Notes.created_at,Notes.body).filter(Notes.key.in_(top_keys)).all()
session.close()

top_docs = pd.DataFrame(result, columns=['key','created_at','body'])

In [ ]:
top_docs.to_csv(f'data/body/{search_word}_{date}_topic{topic_id}_top_body.csv', index=False)